In [22]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors import CellExecutionError
from nbclient.exceptions import CellControlSignal

## Get a list of notebooks to run

In [2]:
import sys
import os
root_path = os.environ.get('NOTEBOOK_ROOT')
sys.path.append(root_path)

# exclude 'AMA', 'experimental', 'IGARSS', and 'legacy' 
#   because these directories are not intended for public release.
# exclude 'DEM' and 'SAR' directories because the testing environment does not have the necessary data.
# exclude 'machine_learning' because we do not have the data
#   to test the Uruguay Random Forest notebooks.
# exclude 'UN_SDG_15_3_1.ipynb' due to no land change data (instructions in nbk).
# exclude 'ALOS_Land_Change.ipynb' due to no ALOS data.
# exclude 'Landslide_Identification_SLIP.iypnb' due to no TERRA ASTER data.
# exclude 'Forest_Change_VNSC.ipynb' due to no TERRA ASTER data.
# exclude 'Shallow_Water_Bathymetry.ipynb' due to no Landsat 8 Level 1 data.
# exclude 'ALOS_WASARD.ipynb' due to no ALOS data.
# exclude 'water_interoperability_similarity.ipynb' due to no Sentinel-2 data.
exclude_subpaths = ['.ipynb_checkpoints', 'AMA', 'DEM', 'SAR', 'experimental', 'IGARSS', 
                    'legacy', 'machine_learning', 'test', 'utils', 
                    'Forest_Change_VNSC.ipynb', 'Shallow_Water_Bathymetry.ipynb',
                    'Automated_Testing.ipynb', 'UN_SDG_15_3_1.ipynb',
                    'ALOS_Land_Change.ipynb', 'Landslide_Identification_SLIP.ipynb',
                    'Forest_Change_VNSC.ipynb', 'Shallow_Water_Bathymetry.ipynb',
                    'ALOS_WASARD.ipynb', 'water_interoperability_similarity.ipynb',
                    ]
notebook_file_paths = []
for root, directories, files in os.walk('..', topdown=True):
    notebook_file_paths.extend([os.path.join(root, file) for file in files if file.endswith('.ipynb')])
notebook_file_paths = sorted(notebook_file_paths)

**Filter out notebooks using `exclude_subpaths`**

In [3]:
def is_filepath_excluded(filepath, exclude_subpaths):
    return any(list(map(lambda subpath: subpath in filepath, exclude_subpaths)))
notebook_file_paths = [filepath for filepath in notebook_file_paths if not is_filepath_excluded(filepath, exclude_subpaths)]

## Run the notebooks and record their status (e.g. working, error) to HTML as each completes

In [4]:
import re
import pandas
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
full_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
error_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
success_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
# Unless this cell is rerun, only run notebooks that have not been run successfully yet.
success_notebooks = []

**To rerun all notebooks, the file `success_nbks_file_name` must be deleted and then this notebook must be restarted**

In [29]:
import pickle

# Load the list of successful notebooks.
success_nbks_file_name = 'success_nbks.pkl'
if os.path.exists(success_nbks_file_name):
    success_nbks_file_in = open(success_nbks_file_name, 'rb')
    success_notebooks = pickle.load(success_nbks_file_in)

notebook_file_paths_to_run = [nbk_pth for nbk_pth in notebook_file_paths if nbk_pth not in success_notebooks]
for notebook_file_path in notebook_file_paths_to_run:
    print(f'Running {notebook_file_path}')
    run_result = {'Notebook': notebook_file_path, 'Status': 'Working', 'Bug Description': ''}
    with open(notebook_file_path, 'r+', encoding='utf-8') as notebook_file:
        notebook = nbformat.read(notebook_file, as_version=4)
        notebook_runner = ExecutePreprocessor(timeout=None)
        try:
            notebook_runner.preprocess(notebook, {'metadata': {'path': os.path.dirname(notebook_file_path)}})
        except CellExecutionError as err:
            run_result['Status'] = 'Error'
            run_result['Bug Description'] = err
        error = run_result['Status'] == 'Error'
        # Save the notebook.
#         nbformat.write(notebook, notebook_file_path)
    full_report = full_report.append(run_result, ignore_index=True)
    if error:
        error_report = error_report.append(run_result, ignore_index=True)
    else:
        # Record that this notebook ran successfully to avoid running it again for this testing session.
        success_notebooks.append(notebook_file_path)
        success_report = success_report.append(run_result, ignore_index=True)
        success_nbks_file_out = open(success_nbks_file_name, 'wb') 
        pickle.dump(success_notebooks, success_nbks_file_out)
    full_report.to_html('full_test_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    error_report.to_html('error_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    success_report.to_html('success_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})

Running ../vegetation/phenology/Vegetation_Phenology.ipynb
Running ../water/detection/water_detection_with_wofs.ipynb
Running ../water/quality/TSM_Demo.ipynb


### Export the results to a CSV

In [ ]:
full_report['Bug Description'] = full_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
full_report.to_csv('full_test_report.csv')
error_report['Bug Description'] = error_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
error_report.to_csv('error_report.csv')
success_report['Bug Description'] = success_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
success_report.to_csv('success_report_report.csv')